<a href="https://colab.research.google.com/github/yndtky/portforio/blob/main/%E3%82%A2%E3%82%A4%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%A0%E3%81%AE%E5%A3%B2%E4%B8%8A%E4%BA%88%E6%B8%AC%E3%81%A8%E6%96%BD%E7%AD%96%E6%8F%90%E6%A1%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

時系列モデルを活用した売上予測を行ないます。

今回はkerasを使って３１アイスクリームの売上高の時系列予測を行ない、予測結果からマーケティング施策の提案を行ってみたいと思います。

この成果物の目的は下記２点です


1.   深層学習フレームワークkerasを使った時系列予測をやってみたかった
2.   予測結果からビジネス視点で何か提案することはできないかと思った




この成果物の流れは次のようになっています。


*   **分析編**
  *   使用データと分析環境
  *   売上データの特徴
  *   作成の流れ
  *   予測結果
  *   分析編の結論
*   **マーケディング編**
  *   アプリ会員者数の推移とこれまでの施策
  *   対象にする顧客層
  *   施策提案
*   **工夫点や苦労点など**





**第一部：分析編**



**使用データと分析環境**

・ここでは31アイスの売上の時系列予測を行なうにあたり、どのようなデータを準備したのかについてまとめます。

・（）の中はデータの取得タイミングと取得元です。

・今回はkerasを勉強するところから始まったことなどもあり、最新の売上予測になっていません。
*   **目的変数**：31アイスの売上データ（四半期ごと、[31アイスHP](https://www.31ice.co.jp/contents/company/ir/index.html)）
*   **説明変数**：
  *   31Club*の会員者数（四半期ごと、[31アイスHP](https://www.31ice.co.jp/contents/company/ir/index.html)）
  *   東京都の消費支出データ（月ごと、[都民のくらしむき（月報）（年報）](https://www.toukei.metro.tokyo.lg.jp/seikei/sb-index.htm) 内の品目別時系列データ）
  *   東京都の最高気温データ（月ごと、[気象庁HP](https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_s3.php?prec_no=44&block_no=47662&year=&month=&day=&view=a2)）
*   データの使用期間：2014年～2023年
*   環境や分析ツールなど：使用言語Python、深層学習フレームワークkerasを使った時系列予測

※31Clubとは31アイスの会員制アプリのことであり、各種クーポンなど様々なサービスが受けられます。


ここで、今回予測する売上データについて少し見てみます。どのような特徴があるでしょうか。

![説明文](main/31_quaterly_sales.png.jpg)


次に、東京都の公開している統計データ（都民のくらしむき（年報）の品目別時系列データ）からアイスクリーム・シャーベットの行を抜き出し、結合してアイスクリーム・シャーベットの消費支出データを作成します。

In [ ]:
#データの前処理
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

folder_path = "/デスクトップ/ファイル/成果物作成/消費データ"

# フォルダ内のすべてのExcelファイルを検索
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# 空のリストを用意して、必要なデータを格納
extracted_data = []

# すべてのExcelファイルを読み込み、条件に合致するデータを抽出してリストに追加
for file in excel_files:
    df = pd.read_excel(file)

    # 左から5番目の列を使用して条件をチェック
    if df.shape[1] > 4:  # 列数が5以上か確認
        condition_rows = df[df.iloc[:, 4] == 'アイスクリーム・シャーベット']

        # 条件に合致する行の左から9列目から20列目の値を抽出し、データフレームとしてリストに追加
        for _, row in condition_rows.iterrows():
            extracted_data.append(row.iloc[8:20].to_frame())
    else:
        print(f"The file {file} does not have enough columns.")

# 抽出したデータフレームを列方向に結合
if extracted_data:
    combined_df = pd.concat(extracted_data, axis=1, ignore_index=True)
else:
    combined_df = pd.DataFrame()
# 最終行を除いたデータフレームを作成
if not combined_df.empty:
    combined_df = combined_df.iloc[:-1, :]
else:
    combined_df = pd.DataFrame()
combined_df.iloc[0,18]=499
# 結果を表示
print(combined_df)